In [0]:
!pip install --force-reinstall puerta_del_sol_paez-0.1.0-py3-none-any.whl

Processing ./puerta_del_sol_paez-0.1.0-py3-none-any.whl
  Attempting uninstall: puerta-del-sol-paez
    Found existing installation: puerta_del_sol_paez 0.1.0
    Not uninstalling puerta-del-sol-paez at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-ce2dc881-2d92-40c3-b638-9aaa145cb4be
    Can't uninstall 'puerta_del_sol_paez'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from puerta_del_sol import limpieza, lectura, funciones

In [0]:
path_bronze = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/'
path_silver = 'abfss://datalake@dls0tfm.dfs.core.windows.net/silver/'
path_input = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/input/'
path_auxiliar = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/auxiliar/'
path_tiempo = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/tiempo/'
path_festivos = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/festivos/'
path_articulos = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/articulos/'
path_vinos = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/vinos/'
path_grupos = 'abfss://datalake@dls0tfm.dfs.core.windows.net/bronze/grupos/'

### Tiempo

In [0]:
df_tiempo = lectura.lee_tiempo(spark, path_input, path_tiempo)
df_tiempo.printSchema

<bound method DataFrame.printSchema of DataFrame[date: timestamp, tavg: double, tmin: double, tmax: double, prcp: double, snow: int, wdir: string, wspd: double, wpgt: double, pres: double, tsun: int]>

In [0]:
df_tiempo = limpieza.limpia_tiempo(df_tiempo)
df_tiempo.printSchema  # PONER CACHE() POR AHÍ
df_tiempo.limit(10).display()

date,tavg,tmin,tmax,prcp,snow,wspd,wpgt,pres,tsun,processing_date
2020-01-01,3.6,1.5,6.5,0.0,0,11.1,29.6,1035.1,338.0,2025-09-04
2020-01-02,-0.6,-3.1,3.6,0.0,0,4.9,14.8,1032.3,452.0,2025-09-04
2020-01-03,-2.2,-3.4,-0.3,0.0,0,5.9,16.7,1027.9,0.0,2025-09-04
2020-01-04,5.8,-2.2,8.3,3.2,0,22.7,72.2,1023.6,5.0,2025-09-04
2020-01-05,2.8,2.1,4.9,0.0,0,20.8,61.1,1032.1,455.0,2025-09-04
2020-01-06,0.5,-3.0,4.4,0.0,0,6.8,22.2,1033.0,385.0,2025-09-04
2020-01-07,0.2,-4.7,4.3,0.0,0,6.3,24.1,1029.1,267.0,2025-09-04
2020-01-08,4.3,1.0,8.6,0.3,0,6.1,18.5,1031.1,449.0,2025-09-04
2020-01-09,2.4,1.9,3.6,0.0,0,7.3,18.5,1025.8,0.0,2025-09-04
2020-01-10,0.2,-0.5,2.1,0.1,0,6.8,18.5,1023.1,47.0,2025-09-04


In [0]:
df_tiempo.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}tiempo')

In [0]:
query_schema = "CREATE SCHEMA IF NOT EXISTS silver"
spark.sql(query_schema)

DataFrame[]

In [0]:
query_table_tiempo = f"""
CREATE TABLE IF NOT EXISTS silver.tiempo
USING DELTA
LOCATION '{path_silver}tiempo' """
spark.sql(query_table_tiempo)

DataFrame[]

### Fechas

In [0]:
df_festivos = lectura.lee_festivos(spark, path_bronze)
df_festivos.printSchema

<bound method DataFrame.printSchema of DataFrame[dia: string, tipo: string]>

In [0]:
df_festivos = limpieza.limpia_festivos(df_festivos)
df_festivos.printSchema

<bound method DataFrame.printSchema of DataFrame[fecha: date, tipo: string]>

In [0]:
fecha_inicio = '2020-01-01'
fecha_fin = '2025-12-31'
df_fechas = funciones.anade_dias(spark, df_festivos, fecha_inicio, fecha_fin)


In [0]:
df_fechas.printSchema

<bound method DataFrame.printSchema of DataFrame[fecha: date, dia_semana: int, mes: int, ano: int, tipo: string, processing_date: date]>

In [0]:
df_fechas.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}fechas')

In [0]:
query_table_fechas = f"""
CREATE TABLE IF NOT EXISTS silver.fechas
USING DELTA
LOCATION '{path_silver}fechas'"""
spark.sql(query_table_fechas)

DataFrame[]

### Artículos

In [0]:
df_art = lectura.lee_articulos(spark, path_input)
df_art.printSchema

<bound method DataFrame.printSchema of DataFrame[*nummer: int, bezeichnung1: string, bezeichnung2: string, bezeichnung3: string, ncode: string, ean: int, preis1: double, preis2: double, preis3: double, preis4: double, preis5: double, preis6: double, preis1frei: string, preis2frei: string, preis3frei: string, preis4frei: string, preis5frei: string, preis6frei: string, wgr: int, gang: int, gruppe: int, drucker1: string, drucker2: string, drucker3: string, drucker4: string, drucker5: string, drucker6: string, drucker7: string, drucker8: string, drucker9: string, drucker10: string, wgrdruck: string, druckfett: string, nachricht: string, beilage: string, kopies: int, Expedit: int, Erloeskonto: int]>

In [0]:
df_art = limpieza.limpia_articulos(df_art)
df_art.printSchema

<bound method DataFrame.printSchema of DataFrame[id: string, name: string, preis1: double, preis2: double, preis3: double, ean: int, wgr: int, processing_date: date]>

In [0]:
df_art.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}articulos')

In [0]:
query_table_articulos = f"""
CREATE TABLE IF NOT EXISTS silver.articulos
USING DELTA
LOCATION '{path_silver}articulos' """
spark.sql(query_table_articulos)

DataFrame[]

### Vinos

In [0]:
df_vinos = lectura.lee_vinos(spark, path_auxiliar)
df_vinos.printSchema

<bound method DataFrame.printSchema of DataFrame[Nombre: string, Tipo: string, Region: string, Uva: string, Seco (trocken): double, Medio: double, Afrutado (fruechtig): double, Lleno (voll): double, Dulce (suess): double, Aromatico (wuerzig): double, Fuerte (kraeftig): double, Medio aromatico medio fuerte: double]>

In [0]:
df_vinos = limpieza.limpia_vinos(df_vinos, df_art)
df_vinos.printSchema

<bound method DataFrame.printSchema of DataFrame[Nombre: string, Tipo: string, Region: string, Uva: array<string>, Seco_trocken: int, Medio: int, Afrutado_fruechtig: int, Lleno_voll: int, Dulce_suess: int, Aromatico_wuerzig: int, Fuerte_kraeftig: int, Medio_aromatico_medio_fuerte: int, id_botella: string, id_vaso: string, processing_date: date]>

In [0]:
df_vinos.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}vinos')

In [0]:
query_table_vinos = f"""
CREATE TABLE IF NOT EXISTS silver.vinos
USING DELTA
LOCATION '{path_silver}vinos' """
spark.sql(query_table_vinos)

DataFrame[]

### Grupos

In [0]:
df_grupos = lectura.lee_grupos(spark, path_input)
df_grupos.printSchema

<bound method DataFrame.printSchema of DataFrame[*nummer: int, bezeichnung1: string, bezeichnung2: string, mwst: int, hgr: int]>

In [0]:
df_grupos = limpieza.limpia_grupos(df_grupos)
df_grupos.printSchema
df_grupos.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}grupos')

In [0]:
query_table_grupos = f"""
CREATE TABLE IF NOT EXISTS silver.grupos
USING DELTA
LOCATION '{path_silver}grupos' """
spark.sql(query_table_grupos)

DataFrame[]

### Ventas

In [0]:
df_ventas = lectura.lee_ventas(spark, path_auxiliar)
df_ventas.printSchema

<bound method DataFrame.printSchema of DataFrame[Nummer:: string, Bezeichnung:: string, Umsatz:: string, Anteil%: string, Menge:: string, date: timestamp]>

In [0]:
df_art = spark.read.format('delta').load(f'{path_silver}articulos')

In [0]:
df_ventas = limpieza.limpia_ventas(df_ventas,df_art)
df_ventas.printSchema

<bound method DataFrame.printSchema of DataFrame[new_id: string, name: string, total_price: double, quantity: int, unity_price: double, date: date, processing_date: date]>

In [0]:
df_ventas.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').partitionBy('processing_date').save(f'{path_silver}ventas')

In [0]:
query_table_ventas = f"""
CREATE TABLE IF NOT EXISTS silver.ventas
USING DELTA
LOCATION '{path_silver}ventas' """
spark.sql(query_table_ventas)

DataFrame[]